In [2]:
import cfscrape
from lxml import etree
import datetime
import os.path

from random import choice, randint 
import time 

user_agents = [
#     'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
#                'Opera/9.25 (Windows NT 5.1; U; en)',
#                'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
#                'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
               'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36', # this is currently enough for all purposes
              ] 

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': choice(user_agents)
         }

In [3]:
def scrape_cloudpiercer_scanid(scan_id):
    df_cloudpierce_output= pd.DataFrame(columns=['tested_domain_name','ip'])

    url="https://cloudpiercer.org/scans/"+scan_id
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content
    html_tree = etree.HTML(scraped_html)
#     print(etree.tostring(html_tree,pretty_print=True)) ##DEBUG

    #Get the tested domain name
    for h1 in html_tree.iter('h1'):
        tested_domain_name = h1.text.split(': ')[1].strip()

    #Get all the IP address from the page
    ip_pattern = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    index=0
    for li in html_tree.iter('li'):
        if re.findall( r'[0-9]+(?:\.[0-9]+){3}', str(li.text)):
#             print(str(li.text)) ##DEBUG
            df_cloudpierce_output.set_value(index, 'tested_domain_name', tested_domain_name)
            df_cloudpierce_output.set_value(index, 'ip', re.findall( r'[0-9]+(?:\.[0-9]+){3}', str(li.text))[0])
            index+= 1        
    return df_cloudpierce_output